# Anomaly Detection

In this notebook, we will explore some basic and advanced anomaly detection techniques. Anomalous data points are those which are significantly different from the rest of the data.

Below you will see some examples of anomaly detection techniques that could be used to find anomalies in different data scenarios.

We will begin with some basic methods that we can use to detect anomalies. Those techniques will focus on descriptive statistics and visualizations of the distribution of the data. For most anomaly detection problems, those simple methods will be insufficient, but it is always a good strategy to begin with simple exploratory methods.

In [ ]:
!pip install pyod
!pip install combo
!pip install pytrends
!pip install statsmodels

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyod.utils.data import generate_data

We will first generate data. We are going to introduce some outliers into the data, so that we know the outliers and their properties. Then we are going to use some simple techniques that may help us finding the outliers in the data. Below we will specify the parameters of the data generating function.

The **contamination** parameter determines the proportion of outlier data points. The **n_features** parameter determines the number of features (columns) of the dataset. Later we will also need to divide the data sample into training and test datasets. But at this stage, we will use the full data sample.

In [ ]:
contamination = 0.05 # share of outliers, here 5%
n_features = 1 # univariate
n_train = 500
n_test = 500

X_train, X_test, y_train, y_test = generate_data(
    n_train = n_train,
    n_test = n_test,
    n_features = n_features,
    contamination = contamination,
    random_state = 123
)

X_train_pd = pd.DataFrame(X_train)
X_train_pd.head(10)

#### Histogram

In [ ]:
plt.hist(X_train_pd[0], bins = 'auto') # bin size definition is important to detect outliers. Small bin sizes make outliers score higher, and vise versa. Auto is usually the best option.
plt.title("X")
plt.show()

#### Boxplot

In [ ]:
plt.boxplot(X_train_pd[0], vert = False)
plt.show()
# Not a very reliable option, if there are too many outliers. Widely used in statistical approaches.

# **Histogram-based Outlier Score**

In [ ]:
This time we are going to generate multivariate data and use a more sophisticated histogram-based method to detect anomalies.

In [ ]:
contamination = 0.05
n_features = 5 #multivariate
n_train = 500
n_test = 500

X_train, X_test, y_train, y_test = generate_data(
    n_train = n_train,
    n_test = n_test,
    n_features = n_features,
    contamination = contamination,
    random_state = 1234
)

X_train_pd = pd.DataFrame(X_train)
X_train_pd.head()

Datapoints can have different outlier score across different dimensions, i.e. it can be seen as an outlier within one dimension, but not others.
Whether a datapoint is an outlier also depends on the domain ones data is from

In [ ]:
plt.scatter(X_train_pd[0], X_train_pd[1], c=y_train, alpha=0.8)
plt.title('Scatter plot')
plt.xlabel('x0')
plt.ylabel('x1')
plt.show()

**Train the model**

In [ ]:
from pyod.models.hbos import HBOS
n_bins = 50
hbos = HBOS(n_bins=n_bins,contamination=0.05)
hbos.fit(X_train)

# Training data
y_train_scores = hbos.decision_function(X_train)
y_train_pred = hbos.predict(X_train)

# Test data
y_test_scores = hbos.decision_function(X_test)
y_test_pred = hbos.predict(X_test) # outlier labels (0 or 1)

# Threshold for the defined comtanimation rate
print("The threshold for the defined contamination rate:" , hbos.threshold_)

def count_stat(vector):
    # Because it is '0' and '1', we can run a count statistic.
    unique, counts = np.unique(vector, return_counts=True)
    return dict(zip(unique, counts))

print("The training data:", count_stat(y_train_pred))
print("The training data:", count_stat(y_test_pred))

In [ ]:
plt.hist(y_train_scores, bins='auto') # arguments are passed to np.histogram
plt.title("Outlier score")
plt.show()
# Observations to the right of the threshold are seen as outliers. In this case the model chose the threshold so that 5% of the data would be to the right of it.

In [ ]:
threshold = hbos.threshold_ # Or other value from the above histogram

def descriptive_stat_threshold(df,pred_score, threshold):
    # Let's see how many '0's and '1's.
    df = pd.DataFrame(df)
    df['Anomaly_Score'] = pred_score
    df['Group'] = np.where(df['Anomaly_Score']< threshold, 'Normal', 'Outlier')

    # Now let's show the summary statistics:
    cnt = df.groupby('Group')['Anomaly_Score'].count().reset_index().rename(columns={'Anomaly_Score':'Count'})
    cnt['Count %'] = (cnt['Count'] / cnt['Count'].sum()) * 100 # The count and count %
    stat = df.groupby('Group').mean().round(2).reset_index() # The avg.
    stat = cnt.merge(stat, left_on='Group',right_on='Group') # Put the count and the avg. together
    return (stat)

descriptive_stat_threshold(X_train,y_train_scores, threshold)

In [ ]:
descriptive_stat_threshold(X_test,y_test_scores, threshold)

In [ ]:
def confusion_matrix(actual,score, threshold):
    Actual_pred = pd.DataFrame({'Actual': actual, 'Pred': score})
    Actual_pred['Pred'] = np.where(Actual_pred['Pred']<=threshold,0,1)
    cm = pd.crosstab(Actual_pred['Actual'],Actual_pred['Pred'])
    return (cm)
confusion_matrix(y_train,y_train_scores,threshold)

In [ ]:
confusion_matrix(y_test,y_test_scores,threshold)

In [ ]:
from pyod.models.combination import aom, moa, average, maximization
from pyod.utils.utility import standardizer
from pyod.models.hbos import HBOS

# Standardize data
X_train_norm, X_test_norm = standardizer(X_train, X_test)

# Test a range of binning
k_list = [5, 10, 15, 20, 25, 30, 50, 60, 75, 100] # Specify a list of bin sizes
n_clf = len(k_list) # calculate based on each bin size. By averaging across different bin sizes the predictive quality can be improved.
# Just prepare data frames so we can store the model results
train_scores = np.zeros([X_train.shape[0], n_clf])
test_scores = np.zeros([X_test.shape[0], n_clf])
# Modeling
for i in range(n_clf):
    k = k_list[i]
    hbos = HBOS(n_bins=n_bins)
    hbos.fit(X_train_norm)
    # Store the results in each column:
    train_scores[:, i] = hbos.decision_function(X_train_norm)
    test_scores[:, i] = hbos.decision_function(X_test_norm)
    thresholds[:, i] = hbos.threshold_
# Decision scores have to be normalized before combination
train_scores_norm, test_scores_norm = standardizer(train_scores,test_scores)

In [ ]:
# Combination by average
# The test_scores_norm is 500 x 10. The "average" function will take the average of the 10 columns. The result "y_by_average" is a single column:
y_train_by_average = average(train_scores_norm)
y_test_by_average = average(test_scores_norm)

import matplotlib.pyplot as plt
plt.hist(y_train_by_average, bins='auto') # arguments are passed to np.histogram
plt.title("Combination by average")
plt.show()

# **Isolation Forest**

In [ ]:
from pyod.models.iforest import IForest
isft = IForest(contamination=0.05, max_samples=40, behaviour='new')
isft.fit(X_train)

# Training data
y_train_scores = isft.decision_function(X_train)
y_train_pred = isft.predict(X_train)

# Test data
y_test_scores = isft.decision_function(X_test)
y_test_pred = isft.predict(X_test) # outlier labels (0 or 1)

# Threshold for the defined comtanimation rate
print("The threshold for the defined contamination rate:" , isft.threshold_)

def count_stat(vector):
    # Because it is '0' and '1', we can run a count statistic.
    unique, counts = np.unique(vector, return_counts=True)
    return dict(zip(unique, counts))

print("The training data:", count_stat(y_train_pred))
print("The training data:", count_stat(y_test_pred))

In [ ]:
isft.get_params()

In [ ]:
isft_vi = isft.feature_importances_
isft_vi

In [ ]:
plt.hist(y_train_scores, bins='auto') # arguments are passed to np.histogram
plt.title("Outlier score")
plt.show()

In [ ]:
threshold = isft.threshold_ # Or other value from the above histogram

def descriptive_stat_threshold(df,pred_score, threshold):
    # Let's see how many '0's and '1's.
    df = pd.DataFrame(df)
    df['Anomaly_Score'] = pred_score
    df['Group'] = np.where(df['Anomaly_Score']< threshold, 'Normal', 'Outlier')

    # Now let's show the summary statistics:
    cnt = df.groupby('Group')['Anomaly_Score'].count().reset_index().rename(columns={'Anomaly_Score':'Count'})
    cnt['Count %'] = (cnt['Count'] / cnt['Count'].sum()) * 100 # The count and count %
    stat = df.groupby('Group').mean().round(2).reset_index() # The avg.
    stat = cnt.merge(stat, left_on='Group',right_on='Group') # Put the count and the avg. together
    return (stat)


In [ ]:
descriptive_stat_threshold(X_train,y_train_scores, threshold)

In [ ]:
descriptive_stat_threshold(X_test,y_test_scores, threshold)

In [ ]:
confusion_matrix(y_train,y_train_scores,threshold)

In [ ]:
confusion_matrix(y_test,y_test_scores,threshold)

In [ ]:
# Standardize data
X_train_norm, X_test_norm = standardizer(X_train, X_test)

# Test a range of maximum samples
k_list = [20, 30, 40, 50, 60]
k_list = [100, 200, 300, 400, 500]
n_clf = len(k_list)
# Just prepare data frames so we can store the model results
train_scores = np.zeros([X_train.shape[0], n_clf])
test_scores = np.zeros([X_test.shape[0], n_clf])

# Modeling
for i in range(n_clf):
    k = k_list[i]
    #isft = IForest(contamination=0.05, max_samples=k)
    isft = IForest(contamination=0.05, n_estimators=k)
    isft.fit(X_train_norm)

    # Store the results in each column:
    train_scores[:, i] = isft.decision_function(X_train_norm)
    test_scores[:, i] = isft.decision_function(X_test_norm)
# Decision scores have to be normalized before combination
train_scores_norm, test_scores_norm = standardizer(train_scores,test_scores)

In [ ]:
# Combination by average
# The test_scores_norm is 500 x 10. The "average" function will take the average of the 10 columns. The result "y_by_average" is a single column:
y_train_by_average = average(train_scores_norm)
y_test_by_average = average(test_scores_norm)
import matplotlib.pyplot as plt
plt.hist(y_train_by_average, bins='auto') # arguments are passed to np.histogram
plt.title("Combination by average")
plt.show()